In [1]:
# coding: utf-8
from PIL import Image
import numpy as np
import time as t
import sys,os
import math
import numpy as np
import sys
from PIL import Image
from sklearn import preprocessing
from sklearn.metrics.pairwise import euclidean_distances
import matplotlib.pyplot as plt
from os import listdir
from os.path import isfile, join
import cv2
from scipy.spatial import distance
import csv
import easydict

In [2]:
##############################################################################
#define parameters
#define which area of points are find to calculate the mean
Jdis = 40
#define the distance threshold for cklusters
bandwidth = 20
threshold = 0.1
proj_path = os.getcwd()
mypath = os.path.join(proj_path,'img')
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
print(len(onlyfiles))
save_path = os.path.join(proj_path,'img_res_meanshift')
if not os.path.exists(save_path):
	os.makedirs(save_path)
dice = []
time_cost = []
##############################################################################

102


In [3]:
##############################################################################
#Ecucidean distance
'''
Method5=Metho1=Method2>Method3

def eudis1(v1, v2):
    return np.linalg.norm(v1-v2)

# 2
def eudis2(v1, v2):
    return distance.euclidean(v1, v2)

# 3
def eudis3(v1, v2):
    return euclidean_distances(v1, v2)

# 5
def eudis5(v1, v2):
    dist = [(a - b)**2 for a, b in zip(v1, v2)]
    dist = math.sqrt(sum(dist))
    return dist
'''

def eudis5(v1, v2):
    dist = [(a - b)**2 for a, b in zip(v1, v2)]
    dist = math.sqrt(sum(dist))
    return dist

In [4]:
##############################################################################
#Dice function
def dice_coefficient(a,b):
    #import pdb;pdb.set_trace()
    #a_bigram_list = [a[i:i+2] for i in range(len(a)-1)]
    #b_bigram_list = [b[i:i+2] for i in range(len(b)-1)]
    a_bigram_list = a
    b_bigram_list = b
    # assignments to save function calls
    lena = len(a_bigram_list)
    lenb = len(b_bigram_list)
    # initialize match counters
    matches = i = j = 0
    while (i < lena and j < lenb):
        if a_bigram_list[i] == b_bigram_list[j]:
            matches += 2
            i += 1
            j += 1
        elif a_bigram_list[i] < b_bigram_list[j]:
            i += 1
        else:
            j += 1
    #import pdb;pdb.set_trace()
    score = float(matches)/float(lena + lenb)
    return score
##############################################################################
#I have another version, and i have pasted the other dice coefficent result in another csv file.
'''
from keras import backend as K
from skimage import img_as_float

def dice_coef(y_true, y_pred, smooth=1):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_coef_loss(y_true, y_pred):
    return -dice_coef(y_true, y_pred)
'''

'\nfrom keras import backend as K\nfrom skimage import img_as_float\n\ndef dice_coef(y_true, y_pred, smooth=1):\n    y_true_f = K.flatten(y_true)\n    y_pred_f = K.flatten(y_pred)\n    intersection = K.sum(y_true_f * y_pred_f)\n    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)\n\ndef dice_coef_loss(y_true, y_pred):\n    return -dice_coef(y_true, y_pred)\n'

In [5]:
##############################################################################
#iterate all files
for inputName in onlyfiles:
	print('Processing image: ',inputName)
	filename, file_extension = os.path.splitext(inputName)
	img = Image.open(os.path.join(mypath,inputName))
	img.load()
	img = np.array(img)
	img_ori = np.array(img)
	seg_img = img
	rows, cols, dim = img.shape
	meandist = np.array([[1000.0 for r in range(cols)] for c in range(rows)])
	labels = np.array([[-1 for r in range(cols)] for c in range(rows)])
	print("Running the Mean Shift algorithm.")
	start = t.time()
	means = []
	for r in range(0,rows,Jdis):
		for c in range(0,cols,Jdis):
			seed = np.array([r,c,img[r][c][0],img[r][c][1],img[r][c][2]])
			x = seed[0]
			y = seed[1]
			r1 = max(0,x-Jdis)
			r2 = min(r1+Jdis*2, rows)
			c1 = max(0,y-Jdis)
			c2 = min(c1+Jdis*2, cols)
			kernel = []
			for i in range(r1,r2):
				for j in range(c1,c2):
					'''
					#import pdb;pdb.set_trace()
					dc = np.linalg.norm(img[i][j] - seed[2:])
					#D = np.linalg.norm([dc,dc])
					dis = np.sqrt(dc*dc*2.0)
					'''
					dis = eudis5(img[i][j],seed[2:])
					if dis < bandwidth:
						kernel.append([i,j,img[i][j][0],img[i][j][1],img[i][j][2]])
			kernel = np.array(kernel)		
			mean = np.mean(kernel,axis=0,dtype=np.int64)
			# Get the shift
			'''
			dc = np.linalg.norm(seed[2:] - mean[2:])
			dis = np.linalg.norm([dc,dc])
			#dis = np.inner(seed[2:] - mean[2:],seed[2:] - mean[2:])
			'''
			dis = eudis5(seed[2:],mean[2:])
			seed = mean
			if dis <= threshold:
				break
			means.append(seed)
	print("Grouping together the means that are closer than the bandwidth.")
	flags = [1 for me in means]
	for i in range(len(means)):
		if flags[i] == 1:
			w = 1.0
			j = i + 1
			while j < len(means):
				'''
				dc = np.linalg.norm(means[i][2:] - means[j][2:])
				#dis = np.linalg.norm([dc,dc])
				dis = np.sqrt(dc*dc*2.0)
				#dis = np.inner(means[i][2:] - means[j][2:],means[i][2:] - means[j][2:])
				'''
				dis = eudis5(means[i][2:],means[j][2:])
				if dis < bandwidth:
					means[i] = means[i] + means[j]
					w = w + 1.0
					flags[j] = 0
				j = j + 1
			means[i] = means[i]/w
	converged_means = []
	for i in range(len(means)):
		if flags[i] == 1:
			converged_means.append(means[i])
	converged_means = np.array(converged_means)
	#print(converged_means.shape)
	print("Constructing the segmented image.")
	#import pdb;pdb.set_trace()
	for i in range(rows):
		for j in range(cols):
			for c in range(len(converged_means)):
				'''
				dc = np.linalg.norm(img[i][j] - converged_means[c][2:])
				#D = np.linalg.norm([dc,dc])
				dis = np.sqrt(dc*dc*2.0)
				'''
				dis = eudis5(img[i][j],converged_means[c][2:])
				if dis < meandist[i][j]:
					meandist[i][j] = dis
					labels[i][j] = c
			seg_img[i][j] = converged_means[labels[i][j]][2:] 

	print("Saving the segmented image.")
	img = (seg_img-np.amin(seg_img))/(np.amax(seg_img)-np.amin(seg_img))*255
	end = t.time()
	img = img.astype(np.uint8)
	#calculate time and dice coeficient
	print("Time taken: " + str((end - start)/60) + " min")
	a = img_ori.reshape(img.shape[0] * img.shape[1]* img.shape[2],1)
	b = img.reshape(img.shape[0] * img.shape[1]* img.shape[2],1)
	dice_dist = dice_coefficient(a,b)
	dice.append(dice_dist)
	print ('Dice similarity score is {}'.format(dice_dist))
	time_cost.append(float(str((end - start)/60)))
	seg_img = Image.fromarray(seg_img)
	seg_img.save("img_res_meanshift/" + filename + "_output.png")


Processing image:  118421_009.png
Running the Mean Shift algorithm.


KeyboardInterrupt: 

In [ ]:
##############################################################################
#Save result
with open('ms_dice.csv', 'w') as fp:
	fp.write(str(dice))
with open('ms_time.csv', 'w') as fw:
	fw.write(str(time_cost))
##############################################################################